 Get Data from Wiki  importing required libraries to get data from wiki,   
 Importing BeautifulSoup to parse the html data,   
 Importing Pandas to work on the dataframe.  

In [2]:
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ------------------------------------------------------------
                         

Getting the Data from wiki page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,"
data = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
output = data.read().decode()

Getting the Data using BeautifulSoup and Declaring Empty Dataframe to Store the values

In [4]:
soup = BeautifulSoup(output, 'html.parser')
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Reading Data from HTML tags and appending to DataFrame   
Assumptions - the table for PostalCode is the first table in the html page   
expected columns in the page in the order of Postcode, Borough , Neighbourhood  

In [5]:
tableSoup = soup.find_all("table")
table = tableSoup[0]
rows = table.find_all("tr")
for row in rows:
    columns = row.find_all("td")
    headers = row.find_all("th")
    #print(len(columns), columns)
    if len(columns) == 0 : continue
    postcode = columns[0].text
    borough = columns[1].text
    neigh = columns[2].text.split("\n")[0]
    neighborhoods = neighborhoods.append({'PostalCode': postcode,
                          'Borough': borough,
                          'Neighborhood': neigh}, ignore_index=True)
        

In [6]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Finding the shape of the dataframe   
finding how many rows are with Borough as 'Not assigned', removing those rows  
and then checking the shape of the resulting dataframe

In [7]:
neighborhoods.shape

(289, 3)

289 Rows in the frame

In [8]:
neighborhoods['Borough'].value_counts()['Not assigned']

77

There are 77 rows where Borough is not Assigned - Filtering those rows

In [9]:
neighDF = neighborhoods[neighborhoods['Borough'].str.contains("[^Not assigned]")==True]

In [10]:
neighDF.shape

(212, 3)

'Not assgined' rows for Borough are filtered successfully 

In [11]:
neighDF.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Checking for rows where Neighbourhood is not assgined and Assigning the value of Borough to Neighborhood

In [12]:
neighDF['Neighborhood'].value_counts()['Not assigned']

1

only one such row found - let's see which rows are those

In [13]:
neighDF[neighDF['Neighborhood'].str.contains("Not assigned")==True]

,PostalCode,Borough,Neighborhood
8,M7A,Queen's Park,Not assigned


row with Neighbourhood as 'Not assgined' is corresponding to "Queen's Park"

In [14]:
neighDF['Neighborhood'].replace(to_replace="Not assigned", value=neighDF['Borough'], inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
neighDF['Neighborhood'].value_counts().get('Not assgined',0)

0

In [16]:
neighDF['Neighborhood'].value_counts().get("Queen's Park",0)

1

Grouping the Values by Postcode and Borough and concatenating the text in Neighbourhood columns

check the shape of unique postcode and borugh values before grouping

In [17]:
len(neighDF['PostalCode'].unique()), len(neighDF['Borough'].unique())

(103, 11)

In [18]:
newDF = pd.DataFrame({'Neighborhood' : neighDF.groupby(by = ['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))}).reset_index()

check the shape of unique postcode and borugh values after grouping

In [19]:
len(neighDF['PostalCode'].unique()), len(neighDF['Borough'].unique())

(103, 11)

In [20]:
newDF.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [21]:
newDF.shape

(103, 3)

Reading data from Geospatial_Data file downloaded from "http://cocl.us/Geospatial_data"

In [22]:
geoDF = pd.read_csv("Geospatial_data")

In [25]:
geoDF.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Three columns in the dataframe merging them with the Toronto PostalCode data

In [23]:
result = pd.concat([newDF, geoDF], axis=1, )

In [24]:
result.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [25]:
resultDF = result.drop('Postal Code', axis = 1)

In [26]:
resultDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
resultDF['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [28]:
len(resultDF['Borough'].unique())

11

there are 11 Boroughs in Toronto    
Let's check number of postal codes in each Borough

In [29]:
for borough in resultDF['Borough'].unique():
    print(borough,len(resultDF.loc[resultDF['Borough'] == borough]['PostalCode'].unique()))

Scarborough 17
North York 24
East York 5
East Toronto 5
Central Toronto 9
Downtown Toronto 18
York 5
West Toronto 6
Queen's Park 1
Mississauga 1
Etobicoke 12


There are 24 PostalCode in the 'North York' Borough - let's compare these postal codes per locality

In [30]:
nyDF = resultDF.loc[resultDF['Borough'] == "North York"]

In [31]:
nyDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
19,M2K,North York,Bayview Village,43.786947,-79.385975
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [32]:
# create map of North York using latitude and longitude values
map_ny = folium.Map(location=[43.73, -79.36], zoom_start=12)

map_ny

In [33]:
# add markers to map
for lat, lng, postal, neighborhood in zip(nyDF['Latitude'], nyDF['Longitude'], nyDF['PostalCode'], nyDF['Neighborhood']):
    label = '{}, {}'.format(postal,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [35]:
# Finding Venues in the radius of 1000m 
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(name, len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

M2L didn't had venues in 500 radius so increased the radius to 1000

In [36]:
# type your answer here

LIMIT = 100
ny_venues = getNearbyVenues(names=nyDF['PostalCode'],
                                   latitudes=nyDF['Latitude'],
                                   longitudes=nyDF['Longitude']
                                  )



M2H
M2H 21
M2J
M2J 42
M2K
M2K 15
M2L
M2L 4
M2M
M2M 34
M2N
M2N 100
M2P
M2P 16
M2R
M2R 10
M3A
M3A 29
M3B
M3B 30
M3C
M3C 41
M3H
M3H 27
M3J
M3J 23
M3K
M3K 18
M3L
M3L 8
M3M
M3M 4
M3N
M3N 31
M4A
M4A 13
M5M
M5M 40
M6A
M6A 48
M6B
M6B 32
M6L
M6L 12
M9L
M9L 10
M9M
M9M 8


In [37]:
ny_venues.head()

,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M2H,43.803762,-79.363452,Tastee,43.807722,-79.356798,Bakery
1,M2H,43.803762,-79.363452,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,43.798391,-79.369187,Korean Restaurant
2,M2H,43.803762,-79.363452,Galati,43.797831,-79.369410,Grocery Store
3,M2H,43.803762,-79.363452,Cummer Park,43.799564,-79.371175,Park
4,M2H,43.803762,-79.363452,Tim Hortons,43.798945,-79.369644,Coffee Shop


In [38]:
ny_venues.shape, len(ny_venues['PostalCode'].unique())

((616, 7), 24)

There are 616 Venues Marked - lets check for unique venues

In [39]:
len(ny_venues['Venue'].unique())

476

In [40]:
len(ny_venues['Venue Category'].unique())

154

In [42]:
nyDF.shape

(24, 5)

In [43]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['PostalCode'] = ny_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,PostalCode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,M2H,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M2H,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


as some locations have very low number of venues and some other high - looking to aggregate them by the number of venues and their categories

In [58]:
ny_grouped =ny_onehot.groupby('PostalCode').sum().reset_index()
ny_grouped.head()

,PostalCode,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Baby Store,Bagel Shop,Bakery,...,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,M2H,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,M2J,0,0,1,0,0,0,0,0,2,...,1,0,0,0,1,0,0,0,0,0
2,M2K,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M2L,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M2M,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0


In [59]:
num_top_venues = 5

for hood in ny_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M2H----
                venue  freq
0                Park   2.0
1            Pharmacy   2.0
2         Coffee Shop   2.0
3      Ice Cream Shop   1.0
4  Chinese Restaurant   1.0


----M2J----
                  venue  freq
0           Coffee Shop   4.0
1        Clothing Store   4.0
2  Fast Food Restaurant   2.0
3   Japanese Restaurant   2.0
4        Sandwich Place   2.0


----M2K----
                 venue  freq
0  Japanese Restaurant   2.0
1                 Bank   2.0
2        Grocery Store   2.0
3             Pharmacy   1.0
4           Skate Park   1.0


----M2L----
               venue  freq
0               Park   3.0
1               Pool   1.0
2  Accessories Store   0.0
3        Pizza Place   0.0
4      Movie Theater   0.0


----M2M----
                       venue  freq
0          Korean Restaurant   4.0
1                       Café   4.0
2  Middle Eastern Restaurant   2.0
3                Pizza Place   2.0
4                Coffee Shop   2.0


----M2N----
                 venue  

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10


indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['PostalCode'] = ny_grouped['PostalCode']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,Park,Pharmacy,Coffee Shop,Bank,Korean Restaurant,Shopping Mall,Sandwich Place,Chinese Restaurant,Diner,Housing Development
1,M2J,Coffee Shop,Clothing Store,Fast Food Restaurant,Bakery,Japanese Restaurant,Sandwich Place,Smoothie Shop,Shopping Mall,Burger Joint,Burrito Place
2,M2K,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Café,Shopping Mall,Skate Park,Skating Rink
3,M2L,Park,Pool,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dog Run
4,M2M,Korean Restaurant,Café,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Ramen Restaurant,Electronics Store,Liquor Store,Sandwich Place,Japanese Restaurant
5,M2N,Japanese Restaurant,Ramen Restaurant,Korean Restaurant,Pizza Place,Coffee Shop,Bubble Tea Shop,Fast Food Restaurant,Café,Restaurant,Sushi Restaurant
6,M2P,Gym,Park,Tennis Court,Coffee Shop,French Restaurant,Pizza Place,Playground,Bank,Golf Course,Intersection
7,M2R,Pharmacy,Shopping Mall,Convenience Store,Bus Line,Coffee Shop,Eastern European Restaurant,Pizza Place,Bakery,Butcher,Yoga Studio
8,M3A,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Fast Food Restaurant,Pizza Place,Coffee Shop,Caribbean Restaurant,Road
9,M3B,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Supermarket,Paper / Office Supplies Store,Diner,Restaurant,Caribbean Restaurant,Salad Place


In [68]:
# set number of clusters
kclusters = 4

ny_grouped_clustering = ny_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:24]

array([2, 0, 2, 2, 3, 1, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2,
       2, 2], dtype=int32)

In [69]:
ny_merged = nyDF

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged =ny_merged.join(ny_venues_sorted.set_index('PostalCode'), on='PostalCode')

ny_merged.head() # check the last columns!

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,2,Park,Pharmacy,Coffee Shop,Bank,Korean Restaurant,Shopping Mall,Sandwich Place,Chinese Restaurant,Diner,Housing Development
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Bakery,Japanese Restaurant,Sandwich Place,Smoothie Shop,Shopping Mall,Burger Joint,Burrito Place
19,M2K,North York,Bayview Village,43.786947,-79.385975,2,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Café,Shopping Mall,Skate Park,Skating Rink
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2,Park,Pool,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dog Run
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,3,Korean Restaurant,Café,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Ramen Restaurant,Electronics Store,Liquor Store,Sandwich Place,Japanese Restaurant


In [70]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[43.73, -79.36], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['PostalCode'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

M2N is the Most Popular Destination for Venues , M2M is another Unique PostalCode

In [71]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Coffee Shop,Clothing Store,Fast Food Restaurant,Bakery,Japanese Restaurant,Sandwich Place,Smoothie Shop,Shopping Mall,Burger Joint,Burrito Place
26,M3B,North York,Don Mills North,43.745906,-79.352188,0,Japanese Restaurant,Burger Joint,Pizza Place,Coffee Shop,Supermarket,Paper / Office Supplies Store,Diner,Restaurant,Caribbean Restaurant,Salad Place
27,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,0,Restaurant,American Restaurant,Asian Restaurant,Gym,Japanese Restaurant,Beer Store,Coffee Shop,Chinese Restaurant,Park,Bus Stop
29,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,0,Pizza Place,Coffee Shop,Restaurant,Furniture / Home Store,Sandwich Place,Japanese Restaurant,Metro Station,Massage Studio,Fast Food Restaurant,Sports Bar
33,M3N,North York,Downsview Northwest,43.761631,-79.520999,0,Hotel,Pizza Place,Pharmacy,Coffee Shop,Grocery Store,Fast Food Restaurant,Kitchen Supply Store,Chinese Restaurant,Caribbean Restaurant,Road
62,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Pub,Pizza Place,Skating Rink,Sandwich Place,Butcher,Café,Restaurant
71,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0,Furniture / Home Store,Coffee Shop,Fast Food Restaurant,Clothing Store,Fried Chicken Joint,Dessert Shop,Vietnamese Restaurant,Hotel,Park,Restaurant
72,M6B,North York,Glencairn,43.709577,-79.445073,0,Grocery Store,Fast Food Restaurant,Coffee Shop,Bus Line,Gym / Fitness Center,Playground,Mediterranean Restaurant,Discount Store,Shoe Store,Pet Store


Cluster 0 is most probably mixed locality which is lined with restaurants and as well Supermarked and General Stores

In [72]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M2N,North York,Willowdale South,43.77012,-79.408493,1,Japanese Restaurant,Ramen Restaurant,Korean Restaurant,Pizza Place,Coffee Shop,Bubble Tea Shop,Fast Food Restaurant,Café,Restaurant,Sushi Restaurant


Cluster 1 which contains only M2N is most probably downtown area with lot of restaurants and cafes lined up

In [73]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,2,Park,Pharmacy,Coffee Shop,Bank,Korean Restaurant,Shopping Mall,Sandwich Place,Chinese Restaurant,Diner,Housing Development
19,M2K,North York,Bayview Village,43.786947,-79.385975,2,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Chinese Restaurant,Pharmacy,Café,Shopping Mall,Skate Park,Skating Rink
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2,Park,Pool,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dog Run
23,M2P,North York,York Mills West,43.752758,-79.400049,2,Gym,Park,Tennis Court,Coffee Shop,French Restaurant,Pizza Place,Playground,Bank,Golf Course,Intersection
24,M2R,North York,Willowdale West,43.782736,-79.442259,2,Pharmacy,Shopping Mall,Convenience Store,Bus Line,Coffee Shop,Eastern European Restaurant,Pizza Place,Bakery,Butcher,Yoga Studio
25,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Convenience Store,Pharmacy,Bus Stop,Shopping Mall,Fast Food Restaurant,Pizza Place,Coffee Shop,Caribbean Restaurant,Road
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,2,Coffee Shop,Bridal Shop,Restaurant,Eastern European Restaurant,Dog Run,Shopping Mall,Diner,Sandwich Place,Middle Eastern Restaurant,Deli / Bodega
30,M3K,North York,"CFB Toronto, Downsview East",43.737473,-79.464763,2,Coffee Shop,Turkish Restaurant,Electronics Store,Italian Restaurant,Pizza Place,Park,Sandwich Place,Café,Soccer Field,Latin American Restaurant
31,M3L,North York,Downsview West,43.739015,-79.506944,2,Park,Bank,Vietnamese Restaurant,Coffee Shop,Moving Target,Grocery Store,Pizza Place,Discount Store,Event Space,Empanada Restaurant
32,M3M,North York,Downsview Central,43.728496,-79.495697,2,Vietnamese Restaurant,Thai Restaurant,Baseball Field,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant


Cluster 2 looks to be residential area as most of the venues are park / stores

In [74]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,3,Korean Restaurant,Café,Coffee Shop,Pizza Place,Middle Eastern Restaurant,Ramen Restaurant,Electronics Store,Liquor Store,Sandwich Place,Japanese Restaurant


Cluster 3 is again lines up with restaurants and eatries